## MVP Prediction Model for buzzerbeater.com

This is a very simple project I worked on for fun for an online basketball manager game at buzzerbeater.com. Most of the work involved came in cleaning the data (see other file) and organizing it to fit a few machine learning models. The models that were fit were basic and have yet to be tuned properly. I will likely engage more with this project in the future.

In [2]:
import pandas as pd
import re
import numpy as np
import warnings
warnings.simplefilter('ignore')

In [3]:
train_data = pd.read_csv("traindataBBMVP.csv")

Not much EDA was performed here. However, the goal is to go back into the other python notebook and look at how data compares across different seasons.

In [4]:
train_data

,Player,GP,MIN,FG,3FG,FT,OR,TR,AST,TO,STL,BLK,PF,PTS,Rating,Win %,PD,MVP Votes
0,Michele Bello,0.954545,33.2,8.7,0.1,2.5,3.6,12.7,3.7,1.3,0.8,1.1,2.3,20.0,18.1,0.682,354.0,192
1,Dotan Ben-David,1.000000,39.0,8.3,0.8,2.2,2.7,12.4,1.8,1.6,0.6,0.9,3.3,19.6,16.3,0.500,96.0,0
2,Ron Fisk,0.772727,33.5,8.2,0.5,2.4,3.4,11.9,1.3,0.8,0.6,0.6,1.4,19.3,16.8,0.136,-642.0,0
3,Donardas Folomejevas,1.000000,38.5,7.7,1.0,2.8,1.0,3.2,4.5,0.8,1.4,0.1,1.5,19.3,16.0,0.591,173.0,40
4,Hvilimon Zianevich,1.000000,26.6,7.8,1.2,2.4,1.2,5.0,1.4,1.1,0.9,0.6,1.7,19.2,14.7,0.727,316.0,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1295,Maks PeÄnik,0.863636,31.8,5.1,0.1,1.7,2.5,11.2,2.4,1.7,0.5,1.0,1.7,11.8,17.7,0.682,366.0,0
1296,Andrew Calvert,0.954545,27.1,4.2,0.7,2.5,1.0,3.5,2.1,0.7,0.5,0.1,2.1,11.7,16.3,0.818,618.0,0
1297,Brent Broussard,1.000000,44.5,4.2,1.4,1.8,1.3,4.1,1.5,2.7,1.0,0.4,2.3,11.6,7.0,0.000,-1608.0,0
1298,Massimo Ciavatta,1.000000,30.1,4.6,0.5,1.8,2.6,9.0,1.9,1.5,0.5,0.8,3.8,11.6,17.1,0.409,152.0,0


In [5]:
X_train = train_data.iloc[:,1:17]
y_train = train_data.iloc[:,17]

Here these links can be replaced with any league from the game to calculate the MVP odds at a certain game # in the season. It shoudl be noted that the current training data is using data from only division 1 leagues.

In [6]:
leagueleaderslink = "https://www.buzzerbeater.com/league/1/stats.aspx"
leagueoverviewlink = "https://www.buzzerbeater.com/league/1/overview.aspx"
n_games = 4

In [7]:
league_leaders = pd.read_html(leagueleaderslink)[10]
leaguestandings1 = pd.read_html(leagueoverviewlink)[10]
leaguestandings2 = pd.read_html(leagueoverviewlink)[11]

In [8]:
league_leaders[1] = league_leaders[1].str[:7]
leaguestandings2['Great 8.1'] = leaguestandings2['Great 8.1'].str[:7]
leaguestandings1['Big 8.1'] = leaguestandings1['Big 8.1'].str[:7]
new_df = pd.merge(league_leaders, leaguestandings2,  how='left', left_on=[1], right_on = ['Great 8.1'])
new_new_df = pd.merge(new_df, leaguestandings1,  how='left', left_on=[1], right_on = ['Big 8.1'])
finalisher_df = new_new_df[[0,            2,            3,
                  4,            5,            6,        7,            8,
                  9,           10,           11,           12,           13,
                 14,           15,        '%_x',
             'PD_x',         '%_y',       'PD_y']]
finalisher_df.rename(columns={0: "Player",            2:"GP",            3:"MIN",
                  4:"FG",            5:"3FG",            6:"FT",        7:"OR",            8:"TR",
                  9:"AST",           10:"TO",           11:"STL",           12:"BLK",           13:"PF",
                 14:"PTS",           15:"Rating",        '%_x':"Win %",
             'PD_x':"PD",         '%_y':"Win % 2",       'PD_y':"PD 2"}, inplace=True)
closer_df = finalisher_df[finalisher_df['Player']!="Player"]
closer_df['Win %'].fillna(closer_df['Win % 2'], inplace=True)
closer_df['PD'].fillna(closer_df['PD 2'], inplace=True)
test_df = closer_df[['Player', 'GP', 'MIN', 'FG', '3FG', 'FT', 'OR', 'TR', 'AST', 'TO',
       'STL', 'BLK', 'PF', 'PTS', 'Rating', 'Win %', 'PD']]
test_df['GP'] = test_df['GP'].astype(float).div(n_games)

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

## Model Training

The models fit are basic and just trained on a simple train test split. I hope to come back in the future and tune them better. Scaling the data based on the season of a player could have a strong impact on improving model performance. RPMSE was the metric looked at the most.

In [10]:
Xtrain1, Xtest1, ytrain1, ytest1 = train_test_split(X_train, y_train, test_size=.20, random_state=100)

In [11]:
lm= LinearRegression()
lm.fit(Xtrain1, ytrain1)
yhatlm = lm.predict(Xtest1)

In [12]:
np.sqrt(np.mean((yhatlm-ytest1)**2))

23.812986078881664

In [13]:
rf= RandomForestRegressor()
rf.fit(Xtrain1, ytrain1)
yhatrf = rf.predict(Xtest1)

In [14]:
np.sqrt(np.mean((yhatrf-ytest1)**2))

19.132418259633

In [67]:
gb= GradientBoostingRegressor()
gb.fit(Xtrain1, ytrain1)
yhatgb = gb.predict(Xtest1)

In [68]:
np.sqrt(np.mean((yhatgb-ytest1)**2))

20.897508949313696

In [69]:
knn = KNeighborsRegressor()
knn.fit(Xtrain1, ytrain1)
yhatknn = knn.predict(Xtest1)

In [70]:
np.sqrt(np.mean((yhatknn-ytest1)**2))

23.648538741651343

Random Forest fit the data the best. Parameters still need to be tuned and cross-validated in the future for better model performance.

In [95]:
rf= RandomForestRegressor()
rf.fit(X_train, y_train)

RandomForestRegressor()

In [16]:
yhatrf = rf.predict(test_df.iloc[:,1:17])

In [17]:
test_df['Expected Points'] = yhatrf

Predictions on my current league at the current time.

In [19]:
test_df.sort_values(by = "Expected Points", ascending = False).head(10)

,Player,GP,MIN,FG,3FG,FT,OR,TR,AST,TO,STL,BLK,PF,PTS,Rating,Win %,PD,Expected Points
23,Christopher Stepp,1.00,36.6,8.0,0.2,1.0,3.8,13.8,4.2,1.5,1.8,1.2,2.5,17.2,16.1,1.00,124.0,85.92
3,Dion Jackson,1.00,38.9,13.2,1.8,2.8,2.2,8.2,1.5,1.2,1.0,0.0,1.0,31.0,17.8,1.00,82.0,76.52
20,Greg Lackey,1.00,30.4,8.0,0.5,1.2,2.0,7.5,3.8,0.2,1.5,0.2,4.0,17.8,16.7,1.00,124.0,75.42
5,Calvin Webb,1.00,37.2,9.8,2.8,2.0,1.2,5.5,3.0,1.0,1.0,0.2,1.8,24.2,17.5,0.50,33.0,60.83
13,Giedrius Liutatanskas,1.00,35.3,7.5,1.8,3.0,0.0,2.2,2.8,0.2,1.0,0.5,2.5,19.8,16.1,1.00,141.0,60.35
8,Ty Maxwell,1.00,40.9,8.2,3.5,2.0,0.2,4.0,4.5,1.5,0.8,0.0,1.2,22.0,17.4,0.25,-35.0,49.29
7,Tanner Knott,1.00,40.2,8.5,1.0,4.2,2.0,8.2,6.5,1.2,2.0,1.0,3.8,22.2,15,0.75,65.0,46.14
46,Philipp Krotzer,1.00,36.3,6.2,0.2,0.5,4.0,17.0,2.0,1.0,0.5,1.0,3.5,13.2,14.7,1.00,141.0,39.52
2,Philip Mrosla,0.75,38.5,13.7,3.3,8.7,2.3,8.0,4.0,0.0,1.0,0.0,2.3,39.3,18.6,0.50,-84.0,35.78
4,Domingo Amarante,1.00,37.6,12.0,4.8,0.8,0.8,6.8,2.0,0.8,0.2,0.0,0.5,29.5,16.8,0.50,33.0,31.36
